In [8]:
%%time
import faiss
import numpy as np
import sys
sys.path.append("/data/home/marialomeli/faiss_improvements/offline_ivf")
from generate_config import read_embeddings
index_path="/fsx-checkpoints/marialomeli/offline_faiss/contriever/github/IVF32768_PQ256.faissindex"
index_ivf=faiss.read_index(index_path) #TODO: need to add to gpu
index_ivf.nprobe = 64
#res = faiss.StandardGpuResources()
#gpu_index_ivf = faiss.index_cpu_to_gpu(res, 0, index_ivf)
# load embeddings
root="/fsx-instruct-opt/swj0419/llama_data/embed/tokenizer-facebook_contriever_msmarco/seq_len-5100/chunk_len-510/model-facebook_contriever_msmarco/"
file_names = [f"github_oss.chunk.{i:02}.jsonl.lz4.npy" for i in range(32)]
file_names=["github_oss.chunk.00.jsonl.lz4.npy"]
d = 768
k=50

for file in file_names:
    fp=root+file
    _,vecs = read_embeddings(fp)
    vecs_small=vecs[:100,:]
    D, I = index_ivf.search(vecs_small, k)

1449570
CPU times: user 5.62 s, sys: 22.8 s, total: 28.5 s
Wall time: 14.4 s


In [3]:
#CPU search
small_time=18.6
num_vecs=100
big_num_vecs=46255545/8
#small_time-num_vecs
#x-big_num_vecs
x=big_num_vecs*small_time/num_vecs
x

1075441.4212500001

In [7]:
1075441.4212500001/3600/24

12.447238671875

In [16]:
#GPU search
small_time=0.040117502212524414
num_vecs=100
big_num_vecs=46255545/8
#small_time-num_vecs
#x-big_num_vecs
x=big_num_vecs*small_time/num_vecs
x

2319.5711610987782

In [6]:
2319.6*8

18556.8

In [2]:
import faiss
import numpy as np
from faiss.contrib.datasets import DatasetBigANN
#this will only work on the fair cluster
data=DatasetBigANN()
count=0
xb=data.database_iterator(bs=1_000)
for i,data_batch in enumerate(xb):
    if i>2:
        break
    print(i)
    filename=f"bigann_small{(i):010}.npy"
    np.save(filename,data_batch)
    print(f"File {filename} is saved!")
    


0
File bigann_small0000000000.npy is saved!
1
File bigann_small0000000001.npy is saved!
2
File bigann_small0000000002.npy is saved!


In [5]:
import numpy as np
test_file=f"/checkpoint/marialomeli/big_ann_data/bigann0000000000.npy"
vec=np.load(test_file,mmap_mode='r')
vec.shape

(50000000, 128)

In [1]:
import faiss
import numpy as np
from faiss.contrib.datasets import DatasetBigANN
#this will only work on the fair cluster
data=DatasetBigANN()
count=0
xq=data.get_queries()
xq.shape

(10000, 128)

In [2]:
import faiss
import numpy as np
index_shard_path="/checkpoint/marialomeli/offline_faiss/bigann/bigann_1B"
queries_path="/checkpoint/marialomeli/big_ann_data/"
# each machine will load an index shard and a query batches and conduct search
# for example
i=0
index_file = index_shard_path+f"/IVF1048576_PQ64.faissindex"
index = faiss.read_index(index_file)
index.by_residual=True
query_batch = np.load(queries_path+f"bigann{(i):010}.npy",mmap_mode='r')

In [4]:
#move index to GPU
res = faiss.StandardGpuResources()
co = faiss.GpuClonerOptions()
co.useFloat16 = True
small_batch=query_batch.shape[0]//8
small_query_batch = query_batch[:small_batch,]
print(small_query_batch.shape)
#50,000/8 per small batch
index_gpu = faiss.index_cpu_to_gpu(res,0,index,co)
D,I = index_gpu.search(query_batch,k=2)
I[0,:]

(6250000, 128)


In [2]:
import os
import numpy as np
def xbin_mmap(fname, dtype, maxn=-1):
    """ mmap the competition file format for a given type of items """
    n, d = map(int, np.fromfile(fname, dtype="uint32", count=2))

    # HACK - to handle improper header in file for private deep-1B
    # if override_d and override_d != d:
    #    print("Warning: xbin_mmap map returned d=%s, but overridig with %d" % (d, override_d))
    #    d = override_d
    # HACK

    assert os.stat(fname).st_size == 8 + n * d * np.dtype(dtype).itemsize
    if maxn > 0:
        n = min(n, maxn)
    return np.memmap(fname, dtype=dtype, mode="r", offset=8, shape=(n, d))


In [5]:
#ssn++ 
# read with https://github.com/harsha-simhadri/big-ann-benchmarks/blob/main/benchmark/dataset_io.py#L94 with dtype=“uint8"
filepath="/datasets01/big-ann-challenge-data/FB_ssnpp/FB_ssnpp_database.u8bin"
dtype="uint8"

data=xbin_mmap(fname=filepath,dtype=dtype)

In [6]:
data.shape

(1000000000, 256)

In [ ]:
data_batch=50000000
for i in range(20):
    if i>2:
        break
    xb_batch=data[i*data_batch:(i+1)*data_batch,:] #[0:49000000],[50000000:99000000],[100000000:149000000]
    print(xb_batch.shape)
    filename=f"/checkpoint/marialomeli/ssnpp_data/ssnpp_{(i):010}.npy"
    np.save(filename,xb_batch)
    print(f"File {filename} is saved!")


In [3]:
# ssnpp benchmarks
# IVF8192_PQ128_np128
IVF8192_PQ128_np128_times=[3887.4,4046.7,4328.8,4519.2,42415.2,7611.4,4477,3951.7,3947.7,3948.5,7611.4,4477.0,3951.7,3947.7,3948.5,3904.1,5906.1,5429.9,5769,4033.9,3893.7,5404.7]
import numpy as np
print("mean:",np.mean(IVF8192_PQ128_np128_times),"std:",np.std(IVF8192_PQ128_np128_times),"min:",min(IVF8192_PQ128_np128_times),"max",max(IVF8192_PQ128_np128_times))


mean: 6427.7863636363645 std: 7930.9687639540025 min: 3887.4 max 42415.2


In [5]:
IVF16384_PQ128_np256_times=[4888.8,4881.7,7581.5,5272.1,5237.8,5110.3,5269.3,5170.5,5304.9,5584.2,4896.1,4873.4,17986.5,7590.9,18021.5, 52393.9]
print("mean:",np.mean(IVF16384_PQ128_np256_times),"std:",np.std(IVF16384_PQ128_np256_times),"min:",min(IVF16384_PQ128_np256_times),"max",max(IVF16384_PQ128_np256_times))


mean: 10003.9625 std: 11720.347793414398 min: 4873.4 max 52393.9


In [6]:
IVF32768_PQ128_np512_times=[12076.9,24584.7,12076.9, 24584.7, 32799.3, 11658.4,9569.8,17956.1,11380.4,29773.2,26810.1,7376.7, 18612.9, 7318.0,7842.5,8334.9,11149.1,8212.9,11840.8,34726.0, 7745]
print("mean:",np.mean(IVF32768_PQ128_np512_times),"std:",np.std(IVF32768_PQ128_np512_times),"min:",min(IVF32768_PQ128_np512_times),"max",max(IVF32768_PQ128_np512_times))


mean: 16020.442857142856 std: 8875.992270247682 min: 7318.0 max 34726.0


In [2]:
import numpy as np
IVF8192_PQ32_np128_times=[5957.5,5966.2,5968.2,5956.6,5969.8,5961,5853.4,5846.2,5929.4,5920.2,5927.5,5950.7,5942.3,5945.4,5949.7,5941.4,5947,5926.7,5953.2,5947.5]
print("mean:",np.mean(IVF8192_PQ32_np128_times),"std:",np.std(IVF8192_PQ32_np128_times),"min:",min(IVF8192_PQ32_np128_times),"max",max(IVF8192_PQ32_np128_times))


mean: 5937.995 std: 32.38220923593702 min: 5846.2 max 5969.8


In [9]:
#online search IVF8192_PQ32_np128
IVF8192_PQ32_np128_online_times={"batch0":8534.295,"batch2":8501.273007392883,"batch5": 8456.743677854538,"batch6": 8520.941766500473,"batch7":8516.610247850418,"batch8": 8461.659716844559,"batch9": 8515.047048330307,"batch10": 8525.11458826065,"batch11":8519.585188150406,"batch12":8452.929758787155,"batch13":8529.626145124435,"batch16":8521.815571546555,"batch17":8457.880843639374,"batch18": 8517.069823026657,"batch19": 8483.311773777008}
print("mean:",np.mean(list(IVF8192_PQ32_np128_online_times.values())),"std:",np.std(list(IVF8192_PQ32_np128_online_times.values())),"min:",min(list(IVF8192_PQ32_np128_online_times.values())),"max",max(list(IVF8192_PQ32_np128_online_times.values())))


mean: 8500.926943805694 std: 28.72199385838948 min: 8452.929758787155 max 8534.295


In [10]:
IVF16384_PQ32_np256_times=[9057.1,9092.2,8969.6,9121.5,9105.7,9128.6,9100.3,9110.4,9092.2,9090.0,9115.2,9115.2,9058.8,9110.1,9146.9,2001.1,2009.7,9134.1,9116.5,2087.2]
print("mean:",np.mean(IVF16384_PQ32_np256_times),"std:",np.std(IVF16384_PQ32_np256_times),"min:",min(IVF16384_PQ32_np256_times),"max",max(IVF16384_PQ32_np256_times))


mean: 8038.120000000001 std: 2523.100273988333 min: 2001.1 max 9146.9
